*please execute the cell below before starting the tutorial by selecting the cell and pressing Ctrl+Enter*

In [ ]:
from aiida.orm import Dict, load_node
from aiida.engine import submit
from aiida import load_profile

load_profile()

# Fleur code

Fleur code is represented by a `FleurCalculation` class in AiiDA.

Generally one needs only two inputs for a Fleur calculation: an `inp.xml` file and initial charge density. As you already know, there is a `FleurinpData` class representing an `inp.xml` file. Therefore, there is a `FleurinpData` object in the inputs. 

No class represents charge density in AiiDA-Fleur and one can not pass it explicitly into a `FleurCalculation`. However, it can be done specifying `RemoteData` node that represents a calculation folder of a previous FLEUR calculation. If a `RemoteData` input is given, the calculation automatically finds its charge density (and `inp.xml` in some cases) and uses it as an input. This will be explained more in next sub-section.

## Inputs of the Fleur calculation

|name|type|description| required |
|:---:|:---:|:---------:|:---:|
|code | Code | Fleur code| yes |
|fleurinpdata | FleurinpData | Object representing inp.xml| no |
|parent_folder | RemoteData | Remote folder of another calculation| no |
|settings | Dict | special settings | no |
|metadata.options | Dict | computational resources | yes |

**code** is a `Code` object representing Fleur code stored in the database; it is the only required input (except `metadata.options`). However, it is not enough to specify the **code** only: one *must* provide one of supported input configurations:

* **code** + **fleurinpdata** : start a calculation from scratch using given `inp.xml`
* **code** + **parent_folder** : start a calculation using `inp.xml` and charge density from the previous calculation
* **code** + **fleurinpdata** + **parent_folder** : start a calculation using `inp.xml` from given `FleurinpData` and charge density from the previous calculation

In this tutorial we are going to use **code** + **fleurinpdata** configuration only.

**settings** is used to modify a list of files to be copied to/from/between calculation folders and to append command options to the executable. We will not cover this in the tutorial.


## Fleur code submission

### Inputs preparation

First of all, we need to import `FleurCalculation` class.

In [ ]:
from aiida_fleur.calculation.fleur import FleurCalculation

or use the `CalculationFactory`:

In [ ]:
from aiida.plugins import CalculationFactory
FleurinputgenCalculation = CalculationFactory('fleur.inpgen')

and prepare input nodes. Configured and ready-to-use Fleur code object is already stored in your database under the PK=???. Now you can load it via:

In [ ]:
fleur_code = load_node(146)

We are going to use a `FleurinpData` that you created in tutorial 4. Please, remind a PK of a Fe monolayer structure and load in via the same command:

In [ ]:
# you need to modify this - please remind the PK of previously created FleurinpData 
fleurinpdata = load_node(PK_IRON_FLEURINPDATA)

We also need to define computational options. We use a parallel Fleur version and thus set 'withmpi' to True and submit two MPI processes:

In [ ]:
options = {'resources' : {"num_machines": 1, "num_mpiprocs_per_machine" : 2},
           'withmpi' : True,
           'max_wallclock_seconds' : 600}

### Assembling inputs in a single dictionary

Similarly to `FleurinpgenCalculation`, we will use `get_builder()` method to assemble all the inputs:

In [ ]:
inputs = FleurCalculation.get_builder()
inputs.code = fleur_code
inputs.fleurinpdata = fleurinpdata
inputs.metadata.options = options

### Job submission

Looks the same as for `FluerinpgenCalculation` (and any other AiiDA calculation):

In [ ]:
fleur_process = submit(FleurCalculation, **inputs)
print('The PK of submitted job is {}'.format(fleur_process.pk))

## Results analysis

After FLEUR code is submitted, you can check the status of all submitted processes for last 24 hours:

In [ ]:
!verdi process list -a -p 1

You can also check the status of inpgen calculation directly. Try running next cell and examine the output:

In [ ]:
# you need to modify this - insert the FleurCalculation PK:
!verdi process status FLEUR_CALC_PK

Wait till the process status becomes 'Finished'. Then, let us explore the output nodes. `FleurCalculation` generates three output nodes:

|name| type | comment|
|:--:|:----:|:----------:|
|output_parameters|Dict| contains parsed `out.xml`|
|remote_folder| FolderData| represents calculation folder |
|retrieved| FolderData | represents retrieved folder |

To extract their PKs, run:

In [ ]:
# you need to modify this - insert the FleurCalculation PK:
!verdi process show FLEUR_CALC_PK

The other way to access the output nodes is to find them in ``fleur_calc_node.outputs``:

In [ ]:
# you need to modify this - insert the FleurCalculation PK:
fleur_calc_node = load_node(FLEUR_CALC_PK)
output_parameters = fleur_calc_node.outputs.output_parameters
print('PK of the output parameters is {}'.format(output_parameters.pk))

The `output_parameters` dictionary contains parsed `out.xml` file. To print its content, run:

In [ ]:
output_parameters.get_dict()

or

In [ ]:
#you need to modify it - replace OUTPUT_PARAM_PK
!verdi data dict show OUTPUT_PARAM_PK

Analysing output dictionary, can you tell what is the band gap and total energy of the ground state?

# Scripting tasks

#### 1.  Run a  Fleur calculation for the Si (remote_folder input configuration)

Create a script and run it in the terminal, passing to `FleurCalculation` **code** + **fleurinpdata** + **remote_folder**. Use **remote_folder** and **fleurinpdata** of the calculation in this tutorial. Modify **fleurinpdata**: change the number of iterations to 30 and mixing parameter alpha to 0.02.

Answer the questions:

1. What files were copied to the calculation folder?
1. How did the total energy change?
2. How did the band gap change?
3. How did the walltime change?